# Ictericia


In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

with open('dados.json', 'r') as f:
    dados = json.load(f)

# Filtro conforme suas regras
dados_filtrados = [d for d in dados if (d["grau_ictericia"] != "estavel") or (d["grau_ictericia"] == d["grau_cianose"])]

X = np.array([d["rgb_ant"] + d["rgb_atu"] for d in dados_filtrados], dtype=np.float32)
y = np.array([d["grau_ictericia"] for d in dados_filtrados])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Configuração do modelo com parâmetros otimizados
rf_clf = RandomForestClassifier(
    n_estimators=150,       # Número de árvores
    max_depth=7,           # Profundidade máxima
    min_samples_split=5,    # Mínimo de amostras para dividir um nó
    class_weight='balanced', # Balanceamento automático de classes
    random_state=42
)

# Treinamento
rf_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Previsões
y_pred = rf_clf.predict(X_test)

# Métricas
print("📊 Relatório de Classificação:")
print(classification_report(y_test, y_pred, zero_division=0))

print("\n📊 Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Extrai importância das features
importances = rf_clf.feature_importances_
features = ['R_ant', 'G_ant', 'B_ant', 'R_atu', 'G_atu', 'B_atu']

# Cria DataFrame para visualização
df_importances = pd.DataFrame({'Feature': features, 'Importance': importances}).sort_values('Importance', ascending=False)

# Plot
plt.figure(figsize=(10,6))
plt.barh(df_importances['Feature'], df_importances['Importance'], color='skyblue')
plt.xlabel('Importância Relativa')
plt.title('Importância das Features no Modelo')
plt.gca().invert_yaxis()  # Mostra a feature mais importante no topo
plt.show()

In [ ]:
# Dados novos (substitua com seus valores reais)
novo_dado = np.array([[180, 150, 140, 160, 120, 90]])  # Formato: [R_ant, G_ant, B_ant, R_atu, G_atu, B_atu]

# Pré-processamento
novo_dado_scaled = scaler.transform(novo_dado)

# Previsão
previsao = rf_clf.predict(novo_dado_scaled)
probabilidades = rf_clf.predict_proba(novo_dado_scaled)

print(f"\n🔮 Previsão para novo dado: {previsao[0]}")
print("Probabilidades por classe:")
for classe, prob in zip(rf_clf.classes_, probabilidades[0]):
    print(f"{classe}: {prob:.2%}")


novo_dado = np.array([[160, 104, 50, 160, 104, 50]])  # Formato: [R_ant, G_ant, B_ant, R_atu, G_atu, B_atu]

# Pré-processamento
novo_dado_scaled = scaler.transform(novo_dado)

# Previsão
previsao = rf_clf.predict(novo_dado_scaled)
probabilidades = rf_clf.predict_proba(novo_dado_scaled)

print(f"\n🔮 Previsão para novo dado: {previsao[0]}")
print("Probabilidades por classe:")
for classe, prob in zip(rf_clf.classes_, probabilidades[0]):
    print(f"{classe}: {prob:.2%}")


In [ ]:
import joblib

# Treinamento do modelo
rf_clf.fit(X_train, y_train)

# Salvando o modelo em um arquivo
joblib.dump(rf_clf, 'rf-ictericia.pkl')

# Cianose

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carrega e filtra os dados
with open('dados.json', 'r') as f:
    dados = json.load(f)

# Filtro específico para cianose (modifique conforme sua necessidade)
dados_filtrados = [d for d in dados if d["grau_cianose"] != "nenhuma"]  # Exemplo: remove casos sem cianose

# Prepara os dados
X = np.array([d["rgb_ant"] + d["rgb_atu"] for d in dados_filtrados], dtype=np.float32)
y_cianose = np.array([d["grau_cianose"] for d in dados_filtrados])  # Rótulos de cianose

# Normalização e divisão
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cianose, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Configuração otimizada para cianose
rf_cianose = RandomForestClassifier(
    n_estimators=200,       # Aumentamos o número de árvores
    max_depth=7,
    min_samples_split=3,    # Mais sensível para padrões sutis de cianose
    class_weight='balanced', # Crucial para cianose que costuma ser mais rara
    random_state=42
)

rf_cianose.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Previsões
y_pred = rf_cianose.predict(X_test)

# Métricas detalhadas
print("📊 Relatório de Classificação para Cianose:")
print(classification_report(y_test, y_pred, zero_division=0))

# Matriz de confusão visual
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=rf_cianose.classes_,
            yticklabels=rf_cianose.classes_)
plt.title('Matriz de Confusão - Cianose')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')
plt.show()

In [ ]:
# Análise específica para padrões de cianose
importances = rf_cianose.feature_importances_
features = ['R_ant', 'G_ant', 'B_ant', 'R_atu', 'G_atu', 'B_atu']

pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values('Importance', ascending=False).plot.barh(x='Feature', y='Importance', color='darkblue')
plt.title('Importância das Features para Cianose')
plt.show()

In [ ]:
def predict_cianose(rgb_ant, rgb_atu):
    """Função completa para previsão de cianose"""
    # Pré-processamento
    novo_dado = np.array(rgb_ant + rgb_atu).reshape(1, -1)
    novo_dado_scaled = scaler.transform(novo_dado)

    # Previsão
    grau = rf_cianose.predict(novo_dado_scaled)[0]
    probas = rf_cianose.predict_proba(novo_dado_scaled)[0]

    # Resultado formatado
    print(f"\n🔵 Resultado para Cianose:")
    print(f"Grau previsto: {grau}")
    print("Probabilidades:")
    for classe, prob in zip(rf_cianose.classes_, probas):
        print(f"• {classe}: {prob:.1%}")

    return grau

# Exemplo de uso:
predict_cianose([184, 176, 172], [150, 127, 139])  # Substitua com valores reais

predict_cianose([160, 104, 50], [160, 104, 50])  # Substitua com valores reais

In [ ]:
import joblib

# Treinamento do modelo
rf_clf.fit(X_train, y_train)

# Salvando o modelo em um arquivo
joblib.dump(rf_cianose, 'rf-cianose.pkl')